In [1]:
# FULLSCREEN APP WITH BACKGROUND IMAGE AND RESPONSIVE PANELS
import ipyvuetify as v
import numpy as np
import pandas as pd
import plotly.express as px
from ipywidgets import widgets, Layout, HTML
from IPython.display import display
from jeodpp import inter, imap

from vois.vuetify import settings
settings.dark_mode      = False
settings.color_first    = '#ff420a'
settings.color_second   = '#ffb98d'
settings.button_rounded = True

from vois.vuetify import app, label, colorPicker, slider, button, toggle, selectSingle, tooltip, switch
from vois import interMap, colors

In [2]:
# Identify polygons without clicking, by showing a tooltip window on the upper-right corner of the map and highlighting the border of the selected polygon on the map
def identifyTooltip(map,p):
    from ipywidgets import widgets, HTML, CallbackDispatcher
    from ipyleaflet import WidgetControl
    from threading import Timer

    message = widgets.HTML(value='')
    widget = WidgetControl(widget=message, position='topright')

    t = None
    pos = [0,0]

    
    # Hide tooltip window and the selected geometry from the map
    def hideTooltip():
        try:
            map.remove_control(widget)
            map.removeLayerByName('Selection')
        except:
            pass
            
    
    # Call the identifyPoint and display the tooltip window
    def displayTooltip():

        sid = inter.identifyPointEx(p,pos[1],pos[0],4326,int(map.zoom))

        updated = False
        if len(sid) > 0:
            vvv = sid.split("#")
            
            # Display geometry
            geom = ''
            if len(vvv) >= 6:
                geom = str(vvv[5])
                
                # Sometimes the closing parentesys is missing (only in Voilà)!
                if geom[:5] == 'MULTI':
                    if geom[-3] != ')':
                        geom += ')'
                elif geom[:5] == 'POLYG':
                    if geom[-2] != ')':
                        geom += ')'
                    
                s = inter.VectorLayer("wkt")
                s.geomAdd(geom)
                s.remove('default','all')
                s.set('line','stroke','#00ffff')
                s.set('line','stroke-width','3')
                map.addLayer(s.toLayer(), name='Selection')
            
            if len(vvv) >= 5:
                message.value = '''
<style>
td {
  border: 1px solid lightgrey;
  padding: 0px;
  border-spacing: 0px;
  margin: 0px;
  font-size: 13px; line-height: 16px;
}
tr {
  font-size: 13px; line-height: 16px;
}
</style>
  <table>
    <tr>
      <td>LAU_NAME</td>
      <td>%s</td>
    </tr>
    <tr>
      <td>YEAR</td>
      <td>%s</td>
    </tr>
    <tr>
      <td><b>Arable land</b></td>
      <td><b>%s%%</b></td>
    </tr>
    <tr>
      <td><b>Permanent Crop</b></td>
      <td><b>%s%%</b></td>
    </tr>
    <tr>
      <td><b>Permanent Grassland</b></td>
      <td><b>%s%%</b></td>
    </tr>
  </table>
''' % (vvv[0],vvv[1],vvv[2],vvv[3],vvv[4])
                try:
                    map.add_control(widget)
                except:
                    pass
                updated = True        
    
        if not updated:
            hideTooltip()
            
    # Callback at every mouse interaction
    def handle_interaction_tooltip(**kwargs):
        nonlocal t, pos
        
        if kwargs.get('type') == 'mouseout':
            hideTooltip()
            if not t is None:
                t.cancel()
           
        elif kwargs.get('type') == 'mousemove':
            pos = [kwargs.get('coordinates')[0],kwargs.get('coordinates')[1]]
            if not t is None:
                t.cancel()
            t = Timer(0.2, displayTooltip)
            t.start()

    map._interaction_callbacks = CallbackDispatcher()
    map.on_interaction(handle_interaction_tooltip)

In [3]:
# Click on the credits text
def on_click_credits():
    g_app.snackbar('CREDITS')
    
# Click on the logo
def on_click_logo():
    g_app.snackbar('LOGO')

# Click on the footer buttons
def on_click_footer(arg):
    g_app.snackbar(arg)

    
# Create the app instance
g_app = app.app(title='Multivariate Choropleth Map example',
                titleheight='6vh',
                titlecredits='Created by Unit I.3 - BDAP',
                titlewidth='44%',
                titlestyle='color: black; font-weight: 500; font-size: 30px;',
                titlespacestyle='width: 1vw;',
                dark=False,
                backgroundimageurl='...',
                backgroundimageposition='bottom left',
                titletabs=[],
                titletabsstile='font-weight: bold; font-size: 16px;',
                titletabscolor='#ca5c1c',
                titletabsdark=True,
                sidepanelwidth=0,
                footercolor=settings.color_second,
                footerdark=False,
                footerbuttons=[],
                footerheight='4vh',
                onclickcredits=on_click_credits,
                onclicklogo=on_click_logo,
                onclickfooter=on_click_footer,
                fullscreen=True)

# Configuration of background image and panels
g_app.contentBackground('...')
g_app.contentResetPanels()

# True if the COntrols panell is enlarged
controlsEnlarged = False

# Reset Controls panel to its default size
def resetControls(name):
    global controlsEnlarged
    
    controlsEnlarged = False
    g_app.contentSetPanel('Controls', '28vw', '44vh', '1vw', '3vh', border='1px solid grey', backcolor='#ffffffff', titlewidth='16.0vh', titleround=False,
                          icon='mdi-menu-down', icontooltip='Enlarge the Controls panel', icononclick=enlargeControls)
    out2.layout.height = '27vh'
    out3.layout.height = '27vh'
    
    g_app.contentSetPanel('Legend', '28vw', '38vh', '1vw', '50.5vh', border='1px solid grey', backcolor='#ffffffff', titlewidth='15.0vh', titleround=False,
                          icon='mdi-menu-down', icontooltip='Hide the Legend panel', icononclick=enlargeControls)
    outLegend.layout.height = '38vh'
    outLegend.clear_output(wait=True)
    with outLegend:
        display(HTML(svg))

    
# Make Controls panel bigger
def enlargeControls(name):
    global controlsEnlarged
    
    controlsEnlarged = True
    g_app.contentSetPanel('Controls', '28vw', '80vh', '1vw','3vh', border='1px solid grey', backcolor='#ffffffff', titlewidth='16.0vh', titleround=False,
                          icon='mdi-menu-up', icontooltip='Reset the Controls panel dimension', icononclick=resetControls)
    out2.layout.height = '54vh'
    out3.layout.height = '54vh'
    
    g_app.contentSetPanel('Legend', '28vw', '2vh', '1vw', '86.5vh', border='0px solid grey', backcolor='#ffffffff', titlewidth='15.0vh', titleround=False,
                          icon='mdi-menu-up', icontooltip='Open the Legend panel', icononclick=resetControls)
    outLegend.clear_output()
    outLegend.layout.height = '0px'

    
    
outControls = g_app.contentAddPanel('28vw', '44vh',   '1vw',  '3vh',    border='1px solid grey', backcolor='#ffffffff', title='Controls', titlewidth='16.0vh', titleround=False, name='Controls',
                                    icon='mdi-menu-down', icontooltip='Enlarge the Controls panel', icononclick=enlargeControls)

outLegend   = g_app.contentAddPanel('28vw', '38vh',   '1vw',  '50.5vh', border='1px solid grey', backcolor='#ffffffff', title='Legend', titlewidth='15.0vh', titleround=False, name='Legend',
                                    icon='mdi-menu-down', icontooltip='Hide the Legend panel', icononclick=enlargeControls)

outMap      = g_app.contentAddPanel('69vw', '85.5vh', '30vw', '3vh',    border='1px solid grey', backcolor='#ffffff00', title='Map',    titlewidth='10.0vh', titleround=False)


# Any change on the widgets
def onchange(arg=None):
    b.setText('Update map and legend (*)')

# Click on the "update" button
def onupdate(arg=None):
    b.setText('Update map and legend')
    updateLegend()

# Click on the "reset" button
def onreset(arg=None):
    s.value  = 5
    c1.color = '#ffeb3b'
    c2.color = '#b71c1c'
    c3.color = '#43a047'
    cb1.color = '#e6a3d0'
    cb2.color = '#8be2ae'
    s1.value = crops[0]
    s2.value = crops[1]
    p1.value = '100'
    p2.value = '100'
    p3.value = '100'
    sw1.value = False
    sw2.value = False
    updateLegend()
    b.setText('Update map and legend')
    
space = v.Html(tag='div',children=[' '], style_='width: 10px; height: 10px; overflow: hidden;')
reset = button.button('Reset', textweight=450, onclick=onreset, width=82, height=42, tooltip='reset to original colors and number of intervals', selected=True, rounded=True)
b = button.button('Update map and legend', textweight=450, onclick=onupdate, width=240, height=42, tooltip='Click to update the map and the legend with the selected intervals and colors', selected=True, rounded=True)

# Controls for the trivariate legend
ln = label.label('Number of intervals:', textweight=450, height=22, tooltip='Select the number of intervals for each of the three variables range', margins=0)
s = slider.slider(5, 2,10, onchange=onchange)

l1 = label.label('Arable land', textweight=450, height=30, tooltip='Select color for Arable land', margins=2)
c1 = colorPicker.colorPicker(color='#ffeb3b', width=44, height=44, rounded=False, onchange=onchange, offset_x=False, offset_y=True)

l2 = label.label('Permanent crop', textweight=450, height=30, tooltip='Select color for Permanent crop', margins=2)
c2 = colorPicker.colorPicker(color='#b71c1c', width=44, height=44, rounded=False, onchange=onchange, offset_x=False, offset_y=True)

l3 = label.label('Permanent grassland', textweight=450, height=30, tooltip='Select color for Permanent grassland', margins=2)
c3 = colorPicker.colorPicker(color='#43a047', width=44, height=44, rounded=False, onchange=onchange, offset_x=False, offset_y=True)

crops = ['Arable land', 'Permanent crop', 'Permanent grassland']

# From attribute description to attribute name
def descr2attribute(attributedescription):
    if   attributedescription == 'Arable land':    return 'AL_PERC'
    elif attributedescription == 'Permanent crop': return 'PC_PERC'
    return 'PG_PERC'

# Controls for the bivariate legend
cb1 = colorPicker.colorPicker(color='#e6a3d0', width=44, height=44, rounded=False, onchange=onchange, offset_x=False, offset_y=True)
cb2 = colorPicker.colorPicker(color='#8be2ae', width=44, height=44, rounded=False, onchange=onchange, offset_x=False, offset_y=True)

s1 = selectSingle.selectSingle('First crop:',  crops, selection=crops[0], width=200, onchange=onchange, clearable=False)
s2 = selectSingle.selectSingle('Second crop:', crops, selection=crops[1], width=200, onchange=onchange, clearable=False)

percentages = ['10', '20', '30', '40', '50', '60', '70', '80', '90', '100']
p1 = selectSingle.selectSingle('High %', percentages, selection=percentages[-1], width=86, onchange=onchange, clearable=False, marginy=1)
p2 = selectSingle.selectSingle('High %', percentages, selection=percentages[-1], width=86, onchange=onchange, clearable=False, marginy=1)
p3 = selectSingle.selectSingle('High %', percentages, selection=percentages[-1], width=86, onchange=onchange, clearable=False, marginy=1)
pt1 = tooltip.tooltip('Set the percentage to be considered as high for the first crop',  p1.draw())
pt2 = tooltip.tooltip('Set the percentage to be considered as high for the second crop', p2.draw())
pt3 = tooltip.tooltip('Set the percentage to be considered as high for the third crop',  p3.draw())

sw1 = switch.switch(False, "4", inset=True, onchange=onchange)
sw2 = switch.switch(False, "4", inset=True, onchange=onchange)
psw1 = tooltip.tooltip('Set the legend on 4 intervals for the first crop',  sw1.draw())
psw2 = tooltip.tooltip('Set the legend on 4 intervals for the second crop', sw2.draw())

maptype = 1    # 0=bivariate    1=trivariate
out2 = widgets.Output(layout=Layout(width='27vw', height='27vh'))
out3 = widgets.Output(layout=Layout(width='27vw', height='27vh'))

# Selection of the type of legend
def ontype(index):
    global maptype
    maptype = index

    onchange()
    
    outControls.clear_output(wait=True)
    with outControls:
        display(space)
        display(t.draw())
        
    # bivariate
    if maptype == 0:
        out2.clear_output(wait=True)
        with out2:
            display(space)
            display(v.Row(no_gutters=True, justify="start",  class_="pa-0 ma-0 ml-6 mt-4", children=[cb1.draw(),space,s1.draw(),space,pt1,space,psw1]))
            display(v.Row(no_gutters=True, justify="start",  class_="pa-0 ma-0 ml-6",      children=[cb2.draw(),space,s2.draw(),space,pt2,space,psw2]))
            display(space)
            display(v.Row(no_gutters=True, justify="start",  class_="pa-0 ma-0 ml-6",      children=[reset.draw()]))
        with outControls:
            display(out2)
    # trivariate
    else:
        out3.clear_output(wait=True)
        with out3:
            display(v.Row(no_gutters=True, justify="start",  class_="pa-0 ma-0 ml-6 mt-3", children=[ln.draw()]))
            display(s.draw())
            display(v.Row(no_gutters=True, justify="start",  class_="pa-0 ma-0 ml-6", children=[pt1,space,c1.draw(),space,l1.draw(), space,reset.draw()]))
            display(v.Row(no_gutters=True, justify="start",  class_="pa-0 ma-0 ml-6", children=[pt2,space,c2.draw(),space,l2.draw()]))
            display(v.Row(no_gutters=True, justify="start",  class_="pa-0 ma-0 ml-6", children=[pt3,space,c3.draw(),space,l3.draw()]))
        with outControls:
            display(out3)
    
    with outControls:
        display(v.Row(no_gutters=True, justify="center", class_="pa-0 ma-0 ml-6 mt-9", children=[b.draw()]))
        

# Selection of the legend type: bi or trivariate
t = toggle.toggle(maptype, ['Bivariate', 'Trivariate'], tooltips=['Define a bivariate choropleth map', 'Define a trivariate choropleth map'],
                  onchange=ontype, row=True, width=160, justify ='center', rounded=False)

ontype(maptype)


# Load data
vector = inter.VectorLayer('wkt')
vector.fileLink('/eos/jeodpp/data/projects/JEODPP/view/mapnik/DEBY_2019_LandCover_LAU.shp','',4326)
vector = vector.parameter("identifyfield", "LAU_NAME YEAR AL_PERC PC_PERC PG_PERC GEOMETRY")
vector = vector.parameter("identifyseparator", "#")

# Create and display the map
m = imap.Map(layout=Layout(width='69vw', height='85.2vh'))
with outMap:
    display(m)

svg = ''

# Creation of the legend
def updateLegend():
    global svg
    
    if controlsEnlarged:
        outLegend.clear_output()
    else:
        outLegend.clear_output(wait=True)
        
    m.clear()
    
    if maptype == 0:  # bivariate
        
        ci1 = colors.colorInterpolator(['#f3f3f3', cb1.color], 0.0, 100.0)
        colorlist1 = [ci1.GetColor(0.0), ci1.GetColor(50.0), ci1.GetColor(100.0)]

        ci2 = colors.colorInterpolator(['#f3f3f3', cb2.color], 0.0, 100.0)
        colorlist2 = [ci2.GetColor(0.0), ci2.GetColor(50.0), ci2.GetColor(100.0)]

        a1 = descr2attribute(s1.value)
        a2 = descr2attribute(s2.value)
        
        h1 = int(p1.value)
        if sw1.value:  # 4 intervals
            intervals1 = ["[%s] < %f"%(a1,h1/4.0), "[%s] >= %f and [%s] < %f"%(a1,h1/4.0,a1,2.0*h1/4.0), "[%s] >= %f and [%s] < %f"%(a1,2.0*h1/4.0,a1,3.0*h1/4.0), "[%s] >= %f"%(a1,3.0*h1/4.0)]
            names1     = ["Very low", "Low", "Medium", "High"]
        else:          # 3 intervals
            intervals1 = ["[%s] < %f"%(a1,h1/3.0), "[%s] >= %f and [%s] < %f"%(a1,h1/3.0,a1,2.0*h1/3.0), "[%s] >= %f"%(a1,2.0*h1/3.0)]
            names1     = ["Low", "Medium", "High"]
            
        h2 = int(p2.value)
        if sw2.value:  # 4 intervals
            intervals2 = ["[%s] < %f"%(a2,h2/4.0), "[%s] >= %f and [%s] < %f"%(a2,h2/4.0,a2,2.0*h2/4.0), "[%s] >= %f and [%s] < %f"%(a2,2.0*h2/4.0,a2,3.0*h2/4.0), "[%s] >= %f"%(a2,3.0*h2/4.0)]
            names2     = ["Very low", "Low", "Medium", "High"]
        else:          # 3 intervals
            intervals2 = ["[%s] < %f"%(a2,h2/3.0), "[%s] >= %f and [%s] < %f"%(a2,h2/3.0,a2,2.0*h2/3.0), "[%s] >= %f"%(a2,2.0*h2/3.0)]
            names2     = ["Low", "Medium", "High"]
            
        svg = interMap.bivariateLegend(vector,
                                       intervals1,
                                       intervals2,
                                       colorlist1,
                                       colorlist2,
                                       title='  ',
                                       title1=s1.value,
                                       names1=names1,
                                       title2=s2.value,
                                       names2=names2,
                                       stroke_width=0.5,
                                       fontsize=22,
                                       fontweight=500,
                                       side=100,
                                       resizewidth='27vw',
                                       resizeheight='34vh')

    else:  # trivariate
        svg = interMap.trivariateLegendEx(vector,
                                          "AL_PERC",
                                          "PC_PERC",
                                          "PG_PERC",
                                          s.value,
                                          0.0,
                                          float(p1.value),
                                          0.0,
                                          float(p2.value),
                                          0.0,
                                          float(p3.value),
                                          color1=c1.color,
                                          color2=c2.color,
                                          color3=c3.color,
                                          color4='#ffffff00',
                                          title='  ',
                                          title1="Arable Land",
                                          title2="Perm. Crop",
                                          title3="Perm. Grassland",
                                          fontsize=20,
                                          fontweight=500,
                                          side=400,
                                          resizewidth='27vw',
                                          resizeheight='34vh',
                                          digits=0,
                                          maxticks=5,
                                          showarrows=True)


    # Display of the vector layer on the map
    p = vector.process()
    m.addLayer(p.toLayer())

    #inter.identifyPopup(m,p)
    identifyTooltip(m,p)

    # Display the trivariate choropleth legend
    if not controlsEnlarged:
        with outLegend:
            display(HTML(svg))
        
    return p

# Fisrt creation of the legend
p = updateLegend()
m.zoomToImageExtent(p)
m.zoom = 7

# Display the default page
g_app.show()

Dialog(children=[Card(children=[Output(layout=Layout(height='100vh', width='100vw'))], layout=None)], fullscre…